In [20]:
import pandas as pd
import numpy as np
from round_3_trader import BS_CALL, implied_volatility, delta
from round_3_trader import Trader

from datamodel import Listing, Observation, Order, OrderDepth, ProsperityEncoder, Symbol, Trade, TradingState
from typing import List, Any
import string
import jsonpickle
import numpy as np
from statistics import NormalDist
import math
import json

In [21]:
class Product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    ARTIFICIAL_PICNIC_BASKET1 = "ARTIFICIAL_PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    SPREAD1 = "SPREAD1"
    VOLCANIC_ROCK = "VOLCANIC_ROCK"
    VOLCANIC_ROCK_VOUCHER_9500 = "VOLCANIC_ROCK_VOUCHER_9500"
    VOLCANIC_ROCK_VOUCHER_9750 = "VOLCANIC_ROCK_VOUCHER_9750"
    VOLCANIC_ROCK_VOUCHER_10000 = "VOLCANIC_ROCK_VOUCHER_10000"
    VOLCANIC_ROCK_VOUCHER_10250 = "VOLCANIC_ROCK_VOUCHER_10250"
    VOLCANIC_ROCK_VOUCHER_10500 = "VOLCANIC_ROCK_VOUCHER_10500"
    



BASKETS_PRODUCTS = {
    Product.PICNIC_BASKET1: 
    {
        Product.CROISSANTS : 6,
        Product.JAMS : 3,
        Product.DJEMBES : 1
    },
    Product.PICNIC_BASKET2:
    {
        Product.CROISSANTS : 4,
        Product.JAMS : 2
    }
}

BASKET1_PRODS = {
    Product.CROISSANTS : 6,
    Product.JAMS : 3,
    Product.DJEMBES : 1
}

BASKET2_PRODS = {
    Product.CROISSANTS : 4,
    Product.JAMS : 2
}


PARAMS = {
    Product.RAINFOREST_RESIN: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 25,
    },
    Product.KELP: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": False,
        "adverse_volume": 15,
        "reversion_beta": 25,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SQUID_INK: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": False,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SPREAD1:{
        "spread_mean": 48.762433333333334,
        "starting_its": 30000,
        "spread_std_window": 25,
        "zscore_threshold": 11,
        "target_position": 60,
    },
    Product.VOLCANIC_ROCK_VOUCHER_9500: {
        "mean_volatility": 0.45,
        "threshold": 0.00163,
        "strike": 9500,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_9750: {
        "mean_volatility": 0.15959997370608378,
        "threshold": 0.00163,
        "strike": 9750,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_10000: {
        "mean_volatility": 0.18,
        "threshold": 0.00163,
        "strike": 10000,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_10250: {
        "mean_volatility": 0.15959997370608378,
        "threshold": 0.00163,
        "strike": 10250,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_10500: {
        "mean_volatility": 0.15959997370608378,
        "threshold": 0.00163,
        "strike": 10500,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
}

Trader1 = Trader()

In [22]:
order_depths = {
	"VOLCANIC_ROCK": OrderDepth(
		buy_orders={10502: 205},
		sell_orders={10504: -146, 10505: -59}
	),
    "VOLCANIC_ROCK_VOUCHER_9500": OrderDepth(
		buy_orders={142: 3, 141: 5},
		sell_orders={144: -5, 145: -8}
	),
    "VOLCANIC_ROCK_VOUCHER_9750": OrderDepth(
		buy_orders={142: 3, 141: 5},
		sell_orders={144: -5, 145: -8}
	),
    "VOLCANIC_ROCK_VOUCHER_10000": OrderDepth(
		buy_orders={505: -19},
		sell_orders={506: -19}
	),
	"VOLCANIC_ROCK_VOUCHER_10250": OrderDepth(
		buy_orders={142: 3, 141: 5},
		sell_orders={144: -5, 145: -8}
	),
    "VOLCANIC_ROCK_VOUCHER_10500": OrderDepth(
		buy_orders={99: 19},
		sell_orders={100: -19}
	),	
}

In [24]:
VOLCANIC_ROCK_VOUCHER_10000_position = 0 
option_orders = Trader1.VolcanicRockCoupon_strat(order_depths = order_depths,prod="VOLCANIC_ROCK_VOUCHER_10000",position=0)
option_orders

[(VOLCANIC_ROCK_VOUCHER_10000, 505, -19)]

In [6]:
VOLCANIC_ROCK_position = 0
underlying_mid_price = Trader1.get_mid_price(order_depths[Product.VOLCANIC_ROCK])
option_mid_price = Trader1.get_mid_price(order_depths[Product.VOLCANIC_ROCK_VOUCHER_10000])

In [10]:



K = Trader1.params[Product.VOLCANIC_ROCK_VOUCHER_10000]['strike']
T = Trader1.params[Product.VOLCANIC_ROCK_VOUCHER_10000]['time to maturity']
r = 0

volatility = implied_volatility(
                option_mid_price,
                underlying_mid_price,
                Trader1.params[Product.VOLCANIC_ROCK_VOUCHER_10000]["strike"],
                T,
                r
            )

delta1 = delta(option_mid_price,K,T,r,volatility)

c:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\round3\round_3_trader.py:17: RuntimeWarning: divide by zero encountered in log
  d1 = (np.log(S/K) + (r - + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
c:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\round3\round_3_trader.py:30: RuntimeWarning: divide by zero encountered in log
  d1 = (np.log(S/K) + (r - + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))


In [17]:
delta1*sum(order.quantity for order in option_orders)

-0.0

In [12]:
volatility

0.505

In [18]:



hedge_orders = Trader1.hedge_option_orders(Product.VOLCANIC_ROCK,order_depths,option_orders,VOLCANIC_ROCK_position, VOLCANIC_ROCK_VOUCHER_10000_position,0.6)


In [19]:
hedge_orders

[(VOLCANIC_ROCK, 10504, 11)]

In [4]:
S = 10502
K1 = 9500
K2 = 9750
K3 = 10000
K4 = 10250
K5 = 10500
T = 5/252
r = 0

V1 = 99.5
V3 = 505

implied_volatility(V3,S,K3,T,r)

0.18391421773478217